In [1]:
import os 
import pathlib
import re
from collections import defaultdict

In [2]:
# path = "../data/Lexis Cases txt/"


The code below answer:

- whether the Plaintiff Wins or not in each case? (T/F)
- The code below iterates over all the text files and gives a list of dictionaries where each dictionary belongs to a text file, keys are the case names and values are whether the plaintiff was successfull (True) ot not (False)

In [925]:
def plaintiff_wins(path):
    '''This function will search the cases and returns a dictionary
    with case names as keys and boolean for value, True if the plaintiff
    wins the case and False if plaintiff looses'''
#     list_of_files = os.listdir(path)
    
    plaintiff_dict = {}
    with open(path,'r') as f:
        contents = f.read() 
        cases = contents.split("End of Document\n")
        for line in cases:
            lines = line.strip().split("\n")
            name = lines[0]        
            #check if it's a British columbia case    
            if "B.C.J" in name:
                if 'R. v.' in name or '(Re)' in name:
                    continue
                
                # regex search for keyword HELD in cases, which determines if case was allowed or dismissed
                HELD = re.search(r'HELD(.+)?', line)
                
                if HELD:
                    matched = HELD.group(0)  
                    liable = re.search(r'(l|L)iab(.+)?.+|(neglige(.+)?)|(convict(.+)?)', matched)
                    dismiss = re.search(r'(dismiss(.+)?.+)|(adjourned.+?)|(negative(.+)?)', matched)
                    damage = re.search(r'(D|d)amage(.+)?.+|(fault(.+)?)', matched)
                    if "allowed" in matched or "favour" in matched or "awarded" in matched or "granted" in matched or "accepted" in matched or "entitled" in matched or "guilty" in matched or liable or damage:
                        plaintiff_dict[name] = "Y"
                    
                    elif dismiss:
                        plaintiff_dict[name] = "N"

                else:
                    if line and name not in plaintiff_dict :
                        last_paras = lines[-5]+" "+lines[-4]+" "+lines[-3]+" "+lines[-2]
                        awarded =  re.search(r'award(.+)?.+?(plaintiff(.+)?)?', last_paras)
                        #regex searches for pattern of plaintiff/defendant/applicant....entitled/have...costs
                        entiteled = re.search(r'(plaintiff|defendant.?|applicant)(.+)?(entitle(.)?(.+)?|have).+?cost(.+)?', last_paras)
                        #regex searches for pattern of successful...(case)
                        successful = re.search(r'successful(.+)?.+?', last_paras)
                        #regex searches for dismiss....
                        dismiss = re.search(r'(dismiss(.+)?.+)|(adjourned.+?)|(negative(.+)?)', last_paras)
                        costs = re.search(r'costs.+?(award(.+)?|cause).+?', last_paras)
                        damage = re.search(r'(D|d)amage(.+)?.+|(fault(.+)?)', last_paras)

                        if dismiss and "not dismissed" not in last_paras:
                            plaintiff_dict[name] = "N"
                        elif damage:
                            plaintiff_dict[name] = "Y"
                        elif awarded:
                            plaintiff_dict[name] = "Y"
                        elif entiteled:
                            plaintiff_dict[name] = "Y"
                        elif successful:
                            plaintiff_dict[name] = "Y"
                        elif costs:
                            plaintiff_dict[name] = "Y"
                        else:
                            plaintiff_dict[name] = "OpenCase"


    return plaintiff_dict

In [951]:
path = "../data/Lexis Cases txt/p79.txt"
plaintiff_dict = plaintiff_wins(path)

In [952]:
for k, val in plaintiff_dict.items():
    if val == "OpenCase":
        print(k,val)


Burnaby (City) v. Intact Insurance Co., [2020] B.C.J. No. 258 OpenCase
Cornester (c.o.b. Millennium Trading) v. Intact Insurance Co., [2012] B.C.J. No. 2439 OpenCase
Law Society of British Columbia v. Robbins, [2014] B.C.J. No. 640 OpenCase
Lopushinsky v. Spraggs, [2000] B.C.J. No. 252 OpenCase
Shen v. Chan, [2016] B.C.J. No. 1570 OpenCase
Summit Leasing Corp. v. Westshore Towing Ltd., [2019] B.C.J. No. 1209 OpenCase
Westbank Lions Seniors Housing v. Scuka Enterprises Ltd., [2013] B.C.J. No. 2553 OpenCase
Carhoun & Sons Enterprises Ltd. v. Canada (Attorney General), [2016] B.C.J. No. 576 OpenCase


In [953]:
len(plaintiff_dict)

49